# 10G Ethernet core example
References:

https://github.com/Xilinx/embeddedsw/blob/master/XilinxProcessorIPLib/drivers/xxvethernet/src/xxxvethernet.c
https://docs.amd.com/r/en-US/pg210-25g-ethernet/Configuration-Register-Map-10G/25G-Ethernet-Subsystem

In [ ]:
import time
import sys
from rfsoc_qsfp_offload.overlay import Overlay

print("Initializing RFSoC 10G Ethernet Offload Overlay")
ol = Overlay(bitfile_name="/opt/bitstream/rfsoc_offload_10g_dummy_stream.bit", ignore_version=True)
#ol = Overlay(ignore_version=True)

# Wait for overlay to initialize
print("Waiting 5 sec for overlay to initialize")
time.sleep(5)
print("Initialized")

### Configure XXV Ethernet Core

In [ ]:
# Stop UDP Stream
ol.udp_stream_0.register_map.USER_RESET = 1

# Reset Ethernet core
eth_mmio = ol.xxv_ethernet_0.mmio
reg_map = ol.xxv_ethernet_0.register_map
tx_cfg_reg = reg_map.CONFIGURATION_TX_REG1
rx_cfg_reg = reg_map.CONFIGURATION_RX_REG1

# Stop Ethernet reciever 
tx_cfg_reg.ctl_tx_enable = 0
rx_cfg_reg.ctl_rx_enable = 0

reg_map.RESET_REG.tx_reset = 1
reg_map.RESET_REG.tx_serdes_reset = 1
reg_map.RESET_REG.rx_reset = 1
reg_map.RESET_REG.rx_serdes_reset = 1

print(f"TX Configuration: {tx_cfg_reg}")
print(f"RX Configuration: {rx_cfg_reg}")

In [ ]:
# Set Ethernet core TX configration
tx_cfg_reg.ctl_tx_enable = 0
reg_map.RESET_REG.tx_reset = 1

# Enable FCS insertion by the TX core. If this bit is set to 0,
# the core does not add FCS to packet. If this bit is set to
# 1, the core calculates and adds the FCS to the packet.
# This input cannot be changed dynamically between
# packets.
tx_cfg_reg.ctl_tx_fcs_ins_enable = 1

# Enable FCS error checking at the AXI4-Stream interface
# by the TX core. This input only has effect when
# ctl_tx_fcs_ins_enable is Low. If this input is Low and a
# packet with bad FCS is being transmitted, it is not
# binned as good. If this input is High, a packet with bad
# FCS is binned as good.
tx_cfg_reg.ctl_tx_ignore_fcs = 0

# Transmit Idle code words. If this input is sampled as a 1,
# the TX path only transmits Idle code words. This input
# should be set to 1 when the partner device is sending
# RFI code words.
tx_cfg_reg.ctl_tx_send_idle = 0 

# Parity error response by the TX Core. If this bit is set to
# 0, the core does not take any action if any parity errors
# are detected. If this bit is set to 1, the core stomps the
# outgoing FCS (i.e., bit-wise inverse) and asserts
# stat_tx_bad_fcs.
tx_cfg_reg.ctl_tx_parity_err_response = 0

reg_map.RESET_REG.tx_reset = 0
reg_map.RESET_REG.tx_serdes_reset = 0
reg_map.RESET_REG.rx_reset = 0
reg_map.RESET_REG.rx_serdes_reset = 0

tx_cfg_reg.ctl_tx_enable = 1
print("Transmit enabled")

rx_cfg_reg.ctl_rx_enable = 1
print("Receive enabled")

# Ethernet core TX status register
print(f"TX_CFG_REG: {tx_cfg_reg}")

# Start UDP Stream
ol.udp_stream_0.register_map.USER_RESET = 0

### Set packet stream rate

In [ ]:
ol.udp_stream_0.register_map.USER_RESET = 1
packet_delay = float(ol.udp_stream_0.register_map.PACKET_DELAY)
print("    Packet delay    : %0f ms" % (packet_delay / 307200))
time.sleep(1)

# 307.2MHz
ol.udp_stream_0.register_map.PACKET_DELAY = int(307.2*30) # 2 Gbps
packet_delay = float(ol.udp_stream_0.register_map.PACKET_DELAY)
print("New Packet delay    : %0f ms" % (packet_delay / 307200))
ol.udp_stream_0.register_map.USER_RESET = 0

In [ ]:
print("Default Dest MAC MSB: %08X" % ol.udp_stream_0.register_map.ETH_DST_MAC_MSB)
print("Default Dest MAC LSB: %08X" % ol.udp_stream_0.register_map.ETH_DST_MAC_LSB)
# Set Destination MAC to MEP-B09E: 6c:92:bf:42:52:12
# Header is updated on MSB write, writes must be in order
#ol.udp_stream_0.register_map.ETH_DST_MAC_LSB = 0x6c92bf42
#ol.udp_stream_0.register_map.ETH_DST_MAC_MSB = 0x00005212

ol.udp_stream_0.register_map.ETH_DST_MAC_LSB = 0xbf425212
ol.udp_stream_0.register_map.ETH_DST_MAC_MSB = 0x00006c92

# Set Destination MAC to broadcast: ff:ff:ff:ff:ff:ff
#ol.udp_stream_0.register_map.ETH_DST_MAC_LSB = 0xffffffff
#ol.udp_stream_0.register_map.ETH_DST_MAC_MSB = 0x0000ffff

print("New Dest MAC MSB    : %08X" % ol.udp_stream_0.register_map.ETH_DST_MAC_MSB)
print("New Dest MAC LSB    : %08X" % ol.udp_stream_0.register_map.ETH_DST_MAC_LSB)

### Check 10G Ethernet core statistics

In [ ]:
# Capture statistics
import time
reg_map.TICK_REG = 1
time.sleep(5)
reg_map.TICK_REG =1 
print("Packets          : %s %s" % (str(reg_map.STAT_TX_TOTAL_PACKETS_MSB),str(reg_map.STAT_TX_TOTAL_PACKETS_LSB)))
print("Small Packets    : %s %s" % (str(reg_map.STAT_TX_PACKET_SMALL_MSB),str(reg_map.STAT_TX_PACKET_SMALL_LSB)))
print("Large Packets    : %s %s" % (str(reg_map.STAT_TX_PACKET_LARGE_MSB),str(reg_map.STAT_TX_PACKET_LARGE_LSB)))
print("65-127           : %s %s" % (str(reg_map.STAT_TX_PACKET_65_127_BYTES_MSB),str(reg_map.STAT_TX_PACKET_65_127_BYTES_LSB)))
print("256-511          : %s %s" % (str(reg_map.STAT_TX_PACKET_256_511_BYTES_MSB),str(reg_map.STAT_TX_PACKET_256_511_BYTES_LSB)))
print("512-1023         : %s %s" % (str(reg_map.STAT_TX_PACKET_512_1023_BYTES_MSB),str(reg_map.STAT_TX_PACKET_512_1023_BYTES_LSB)))
print("1024-1518        : %s %s" % (str(reg_map.STAT_TX_PACKET_1024_1518_BYTES_MSB),str(reg_map.STAT_TX_PACKET_1024_1518_BYTES_LSB)))
print("Total bytes      : %s %s" % (str(reg_map.STAT_TX_TOTAL_BYTES_MSB),str(reg_map.STAT_TX_TOTAL_BYTES_LSB)))
print("Good bytes       : %s %s" % (str(reg_map.STAT_TX_TOTAL_GOOD_BYTES_MSB),str(reg_map.STAT_TX_TOTAL_GOOD_BYTES_LSB)))
print("FIFO Error       : %s %s" % (str(reg_map.STAT_TX_FRAME_ERROR_MSB),str(reg_map.STAT_TX_FRAME_ERROR_LSB)))
print("Frame Error      : %s %s" % (str(reg_map.STAT_TX_FRAME_ERROR_MSB),str(reg_map.STAT_TX_FRAME_ERROR_LSB)))
print("FCS   Error      : %s %s" % (str(reg_map.STAT_TX_BAD_FCS_MSB),str(reg_map.STAT_TX_BAD_FCS_LSB)))
print("Unicast Packets  : %s %s" % (str(reg_map.STAT_TX_UNICAST_MSB),str(reg_map.STAT_TX_UNICAST_LSB)))
print("Broadcast Packets: %s %s" % (str(reg_map.STAT_TX_BROADCAST_MSB),str(reg_map.STAT_TX_BROADCAST_LSB)))

### Configure UDP Streaming core
Stop here if using default values, configuration changes may break stream to receiving devices

Enable USER_RESET before changing any configuration values

In [ ]:
ol.udp_stream_0.register_map.USER_RESET = 1

In [ ]:
print("Default Destination IP: %08X" % ol.udp_stream_0.register_map.IP_DST_ADDR)
#192.168.4.1 (default)
ol.udp_stream_0.register_map.IP_DST_ADDR = 0xc0A80401
print("New Destination IP    : %08X" % ol.udp_stream_0.register_map.IP_DST_ADDR)

In [ ]:
print("Default Source IP: %08X" % ol.udp_stream_0.register_map.IP_SRC_ADDR)
#192.168.4.8
ol.udp_stream_0.register_map.IP_SRC_ADDR = 0xc0A80408
print("New Source IP    : %08X" % ol.udp_stream_0.register_map.IP_SRC_ADDR)


In [ ]:
print("Default Source PORT: %08X" % ol.udp_stream_0.register_map.IP_SRC_PORT)
#1020
ol.udp_stream_0.register_map.IP_SRC_PORT = 1020
print("New Source PORT    : %08X" % ol.udp_stream_0.register_map.IP_SRC_PORT)


In [ ]:
# 5-bit address bus?
print("Default Destination PORT: %08X" % ol.udp_stream_0.register_map.IP_DST_PORT)
#1020
ol.udp_stream_0.register_map.IP_DST_PORT = 1021
print("New Destination PORT    : %08X" % ol.udp_stream_0.register_map.IP_DST_PORT)


In [ ]:
ol.udp_stream_0.register_map.USER_RESET = 0